In [1]:
!pip uninstall opencv-python

Found existing installation: opencv-python 4.1.2.30
Uninstalling opencv-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/cv2/*
    /usr/local/lib/python3.7/dist-packages/opencv_python-4.1.2.30.dist-info/*
Proceed (y/n)? y
  Successfully uninstalled opencv-python-4.1.2.30


In [2]:
!pip install opencv-python

     |████████████████████████████████| 60.4 MB 1.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [3]:
#Checking version of OpenCV
import cv2
cv2.__version__

'4.5.5'

In [4]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [5]:
%cd '/content/gdrive/MyDrive/Computer_Vision_Docs/Object_Tracking/Yolo_Object_Tracking/'


/content/gdrive/MyDrive/Computer_Vision_Docs/Object_Tracking/Yolo_Object_Tracking


In [6]:
import numpy as np
import math

from google.colab.patches import cv2_imshow


In [7]:
#Define Class ObjectDetection with Yolo4
class ObjectDetection:
    def __init__(self, weights_path="dnn_model/yolov4.weights", cfg_path="dnn_model/yolov4.cfg"):
        print("Loading Object Detection")
        print("Running opencv dnn with YOLOv4")
        self.nmsThreshold = 0.4
        self.confThreshold = 0.5
        self.image_size = 608  #Tùy chỉnh

        # Load Deep Neuro Network (DNN)
        net = cv2.dnn.readNet(weights_path, cfg_path)

        # Set 'DNN' to CUDA
        net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
        self.model = cv2.dnn_DetectionModel(net)

        self.classes = []
        self.load_class_names()
        self.colors = np.random.uniform(0, 255, size=(80, 3))

        self.model.setInputParams(size=(self.image_size, self.image_size), scale=1/255)

    def load_class_names(self, classes_path="dnn_model/classes.txt"):

        with open(classes_path, "r") as file_object:
            for class_name in file_object.readlines():
                class_name = class_name.strip()
                self.classes.append(class_name)

        self.colors = np.random.uniform(0, 255, size=(80, 3)) #Tạo color cho 80 lớp đối tượng trong Yolo4
        return self.classes

    def detect(self, frame):  #Object Detection cho từng frame
        return self.model.detect(frame, nmsThreshold=self.nmsThreshold, confThreshold=self.confThreshold)



In [19]:
END_POINT = 700  # the height of the drawing line from the origin of the image frame
LINE_X_START_POINT = 300  # the x value of the starting point of the line
LINE_X_END_POINT = 1250   # the x value of the ending point of the line

In [9]:
#Định nghĩa hàm check_location để đếm cars
def check_location(center_point):
    flag = False
    if (center_point[0] > LINE_X_START_POINT and center_point[0] < LINE_X_END_POINT):
      if center_point[1] > END_POINT:
          flag = True
          
    return flag


In [11]:
# Initialize Object Detection
od = ObjectDetection()

Loading Object Detection
Running opencv dnn with YOLOv4


In [20]:
#Open the input video clip

cap = cv2.VideoCapture("./high_way.mp4")


In [21]:
# Initialize count, tracking objects and tracking id
count = 0
center_points_prev_frame = []

tracking_objects = {}
track_id = 0
number_vehicle = 0

In [22]:
#Get the width, height of the frame
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
res=(int(width), int(height))

#Define the configuration of the output video
fourcc = cv2.VideoWriter_fourcc(*'MP4V') #format of the output video
out = cv2.VideoWriter('output_high_way_counting_cars.mp4', fourcc, 30.0, res)

last_frame = []
while True:
    
    ret, frame = cap.read()
    count += 1

    if not ret:
        break

    
    # Current frame point
    center_points_cur_frame = []

    # Detect objects on frame
    (class_ids, scores, boxes) = od.detect(frame) 
    for box in boxes:
        (x, y, w, h) = box
        cx = int((x + x + w) / 2)
        cy = int((y + y + h) / 2)

        center_points_cur_frame.append((cx, cy))
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # OBJECT TRACKING
    # Only at the beginning we compare previous and current frame
    if count <= 2:
        for pt in center_points_cur_frame:
            for pt2 in center_points_prev_frame:
                distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])

                if distance < 20:
                    tracking_objects[track_id] = pt
                    track_id += 1
    else:

        tracking_objects_copy = tracking_objects.copy()
        center_points_cur_frame_copy = center_points_cur_frame.copy()

        for object_id, pt2 in tracking_objects_copy.items():
            object_exists = False
            for pt in center_points_cur_frame_copy:
                distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])

                # Update IDs position
                if distance < 20:
                    tracking_objects[object_id] = pt
                    object_exists = True
                    if pt in center_points_cur_frame:
                        center_points_cur_frame.remove(pt)
                    continue

            # Remove IDs lost (out of the frame)
            if not object_exists:
                tracking_objects.pop(object_id)

        # Add new IDs found và loại ra những object đã đếm
        for pt in center_points_cur_frame:
          if not check_location(pt):
            tracking_objects[track_id] = pt
            track_id += 1


        #REMOVE IDs when passing the line
        tracking_objects_copy = tracking_objects.copy()
        for object_id, pt in tracking_objects_copy.items():
            if check_location(pt):
              number_vehicle += 1
              tracking_objects.pop(object_id)


    for object_id, pt in tracking_objects.items():
        cv2.circle(frame, pt, 5, (0, 0, 255), -1)
        cv2.putText(frame, str(object_id), (pt[0], pt[1] - 7), 0, 1, (0, 0, 255), 2)


    # Put summary text for number of counted cars
    cv2.putText(frame, "Number : {:03d}".format(number_vehicle), (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 2)

    cv2.line(frame, (LINE_X_START_POINT, END_POINT), (LINE_X_END_POINT, END_POINT), (0,255,255), 2)
    
    # Make a copy of the points
    center_points_prev_frame = center_points_cur_frame.copy()

    # Write an output video
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    out.write(image)

    last_frame = frame

    
    if count % 10 == 0:
      print('Count:', count)


    if cv2.waitKey(30) & 0xff == ord('q'):
        break

#Print the last frame when completing
cv2_imshow(last_frame)

out.release()

cap.release()
cv2.destroyAllWindows()

Output hidden; open in https://colab.research.google.com to view.